# Run process for one user inputing their results

In [56]:
import pandas as pd
import numpy as np
import psycopg2
import pymongo
import json
import datetime
import pickle
import functions as fn
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import io
from sqlalchemy import create_engine
import itertools
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import emoji
import regex
import nltk
from nltk import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords, wordnet
import string
from emoji.unicode_codes import UNICODE_EMOJI as ue
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.cluster import cosine_distance
from nltk.cluster.kmeans import KMeansClusterer
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora
import itertools

## Gather the necessary features from them

In [32]:
when_did_you_open_your_account = datetime.datetime.strptime(input("Date when the account was opened: "), '%d %B %Y')

Date: 10 June 2016


In [33]:
time_since_account_inception = when_did_you_open_your_account - datetime.datetime.today()

In [34]:
n_transactions_made_last_week = int(input("Number of transactions made last week: "))

Number: 5


In [35]:
how_many_were_during_the_week_end = int(input("How many of those were during the weekend: "))

Number: 2


In [36]:
trans_made_in_week = n_transactions_made_last_week - how_many_were_during_the_week_end

In [42]:
how_many_were_yesterday = int(input("How many of those were done yesterday: "))

How many of those were done yesterday: 1


In [29]:
n_transactions_made_to_diff_users = int(input("How many of those were to different users: "))

3


In [37]:
max_time_between_transactions = (
    int(input("What was the highest number of days that went by between two transactions: ")) * 3600 * 24)

How many days went by before you made a transaction: 2


In [38]:
average_time_between_transactions = int(input("How frequent where the transactions (hours) : ")) * 3600

How frequent where the transactions (hours) : 12


In [39]:
trans_pending_of_those_made = int(input("How many of them failed : "))

How many of them failed : 0


In [41]:
text_description = input("Favourite bank transfer description: ")

Favourite bank transfer description: Money for drugs


## Create dataframe with only the required features

In [60]:
def get_clean_text_pattern(recomposed_note):
    """Function that filters through the notes, retrieves those that match
     the specified pattern and removes stopwords."""
    pattern = "([a-zA-Z0-9\\\]+(?:'[a-z]+)?)"
    recomposed_note_raw = nltk.regexp_tokenize(recomposed_note, pattern)
    # Create a list of stopwords and remove them from our corpus
    stopwords_list = stopwords.words('english')
    stopwords_list += list(string.punctuation)
    # additional slang and informal versions of the original words had to be added to the corpus.
    stopwords_list += (["im", "ur", "u", "'s", "n", "z", "n't", "brewskies", "mcd’s", "Ty$",
                        "Diploooooo", "thx", "Clothessss", "K2", "B", "Comida", "yo", "jobby",
                        "F", "jus", "bc", "queso", "fil", "Lol", "EZ", "RF", "기프트카드", "감사합니다",
                        "Bts", "youuuu", "X’s", "bday", "WF", "Fooooood", "Yeeeeehaw", "temp",
                        "af", "Chipoodle", "Hhuhhyhy", "Yummmmers", "MGE", "O", "Coook", "wahoooo",
                        "Cuz", "y", "Cutz", "Lax", "LisBnB", "vamanos", "vroom", "Para", "el", "8==",
                        "bitchhh", "¯\\_(ツ)_/¯", "Ily", "CURRYYYYYYY", "Depósito", "Yup", "Shhhhh"])

    recomposed_note_stopped = ([w.lower() for w in recomposed_note_raw if w not in stopwords_list])
    return recomposed_note_stopped

In [64]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [71]:
def lemmatize_notes(recomposed_note_stopped):
    "Function that lemmatizes the different notes."
    # Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    lemmatized_notes = []
    for sentence in recomposed_note_stopped:
        for word in nltk.word_tokenize(sentence):
            lem = lemmatizer.lemmatize(word, get_wordnet_pos(word))
            lemmatized_notes.append(lem)
    return lemmatized_notes

In [158]:
def get_user_df(when_did_you_open_your_account, n_transactions_made_last_week, how_many_were_during_the_week_end,
                how_many_were_yesterday, n_transactions_made_to_diff_users, trans_pending_of_those_made,
                max_time_between_transactions, average_time_between_transactions, text_description):
    "Function that returns a specific user dataframe with the relevant inputs"
    user_details = {}
    user_details['time_since_account_inception'] = (
        datetime.datetime.today() - when_did_you_open_your_account).total_seconds()
    user_details['n_transactions_made_last_week'] = n_transactions_made_last_week
    user_details['n_transactions_made_during_weekend'] = how_many_were_during_the_week_end
    user_details['n_transactions_made_during_week'] = (
        n_transactions_made_last_week - how_many_were_during_the_week_end)
    user_details['n_transactions_made_yesterday'] = how_many_were_yesterday
    user_details['n_transactions_made_to_diff_users'] = n_transactions_made_to_diff_users
    user_details['max_time_between_transactions'] = max_time_between_transactions
    user_details['mean_time_between_transactions'] = average_time_between_transactions   
    user_details['unsuccesful_transactions'] = trans_pending_of_those_made  
    
    user_details_df = pd.DataFrame([user_details])
    
    # Dealing with the text aspect
    recomposed_note_stopped = get_clean_text_pattern(text_description)
    lemmatized_notes = lemmatize_notes(recomposed_note_stopped)
    # Load the vectorizer model
    vectorizer= Doc2Vec.load("d2v.model")
    # Find the vectors for each note in the whole note corpus
    _vectrs = [np.array(vectorizer.infer_vector(lemmatized_notes))]
    _vectrs_df = pd.DataFrame(_vectrs)
    
    user_details_combined = pd.concat([user_details_df, _vectrs_df], axis=1)
    return user_details_combined

In [160]:
user_details = get_user_df(when_did_you_open_your_account, n_transactions_made_last_week, how_many_were_during_the_week_end,
                how_many_were_yesterday, n_transactions_made_to_diff_users, trans_pending_of_those_made,
                max_time_between_transactions, average_time_between_transactions, text_description)

/Users/jjherranzsarrion/anaconda3/envs/learn-env/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Predict transactions for a given user

In [154]:
# load the model from disk
red_ent_forest_model = pickle.load(open('red_ent_forest.sav', 'rb'))

In [162]:
ent_forest_y_hat_preds = red_ent_forest_model.predict_proba(user_details)

In [163]:
ent_forest_y_hat_preds

array([[0.85855689, 0.14144311]])